In [1]:
import os
from PIL import Image
import numpy as np

openslide_path = r'C:\Users\Vasco\Documents\openslide-win64-20231011\bin'

os.environ['PATH'] = openslide_path + ";" + os.environ['PATH']
# from openslide import OpenSlide


if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(openslide_path):
        import openslide
else:
    import openslide
from openslide import OpenSlide

You need to do get crop data before running this code. I use a matlab function I made with the 2.5x or 10x tiff images.

Function is here: \\10.99.68.178\andreex\students\Donald Monkey fetus\codes\monkey\crop1024_monkey.m

In [2]:
pth = r'\\10.99.68.178\andreex\students\Vasco Fallopian tube\stardist'      # path to ndpis
pth_mat = r'\\10.99.68.178\andreex\students\Vasco Fallopian tube\stardist\1x\matfiles'               # path to matfiles

In [3]:
# Get a list of MAT files and their base names (without extensions)
mat_files = [os.path.splitext(file)[0] for file in os.listdir(pth_mat) if file.endswith('.mat')]

# Get a list of NDPI images and their base names (without extensions)
ndpi_files = [os.path.splitext(file)[0] for file in os.listdir(pth) if file.endswith('.ndpi')]

# Find the matching files based on their base names
matching_files = set(mat_files).intersection(ndpi_files)

WSIs = sorted([filename + '.ndpi' for filename in matching_files])

crop_mats = sorted([filename + '.mat' for filename in mat_files])

In [4]:
crop_mats

['fallopian_tubes_AJER376_035.mat',
 'fallopian_tubes_AJER376_211.mat',
 'fallopian_tubes_AJER376_235.mat',
 'fallopian_tubes_AJER376_245.mat',
 'fallopian_tubes_AJER376_431.mat',
 'fallopian_tubes_AJGB283_155.mat',
 'fallopian_tubes_AJGB283_487.mat',
 'fallopian_tubes_AJLC170_0153.mat',
 'fallopian_tubes_AJLC170_0729.mat',
 'fallopian_tubes_AJLF232_0153.mat',
 'fallopian_tubes_AJLF232_0483.mat']

In [5]:
WSIs

['fallopian_tubes_AJER376_035.ndpi',
 'fallopian_tubes_AJER376_211.ndpi',
 'fallopian_tubes_AJER376_235.ndpi',
 'fallopian_tubes_AJER376_245.ndpi',
 'fallopian_tubes_AJER376_431.ndpi',
 'fallopian_tubes_AJGB283_155.ndpi',
 'fallopian_tubes_AJGB283_487.ndpi',
 'fallopian_tubes_AJLC170_0153.ndpi',
 'fallopian_tubes_AJLC170_0729.ndpi',
 'fallopian_tubes_AJLF232_0153.ndpi',
 'fallopian_tubes_AJLF232_0483.ndpi']

In [6]:
res20x=0.4416
res2_5x=8

ratio = res2_5x/res20x


outpth = os.path.join(pth,'tiles512')
if not os.path.exists(outpth):
    os.mkdir(outpth)

In [7]:
print(ratio)

18.115942028985508


In [8]:
outpth = r'\\10.99.68.178\andreex\students\Vasco Fallopian tube\stardist\tiles512'
outpth

'\\\\10.99.68.178\\andreex\\students\\Vasco Fallopian tube\\stardist\\tiles512'

In [9]:
crop_mats

['fallopian_tubes_AJER376_035.mat',
 'fallopian_tubes_AJER376_211.mat',
 'fallopian_tubes_AJER376_235.mat',
 'fallopian_tubes_AJER376_245.mat',
 'fallopian_tubes_AJER376_431.mat',
 'fallopian_tubes_AJGB283_155.mat',
 'fallopian_tubes_AJGB283_487.mat',
 'fallopian_tubes_AJLC170_0153.mat',
 'fallopian_tubes_AJLC170_0729.mat',
 'fallopian_tubes_AJLF232_0153.mat',
 'fallopian_tubes_AJLF232_0483.mat']

In [10]:
WSIs

['fallopian_tubes_AJER376_035.ndpi',
 'fallopian_tubes_AJER376_211.ndpi',
 'fallopian_tubes_AJER376_235.ndpi',
 'fallopian_tubes_AJER376_245.ndpi',
 'fallopian_tubes_AJER376_431.ndpi',
 'fallopian_tubes_AJGB283_155.ndpi',
 'fallopian_tubes_AJGB283_487.ndpi',
 'fallopian_tubes_AJLC170_0153.ndpi',
 'fallopian_tubes_AJLC170_0729.ndpi',
 'fallopian_tubes_AJLF232_0153.ndpi',
 'fallopian_tubes_AJLF232_0483.ndpi']

In [11]:
from scipy.io import loadmat
import h5py

file_format = '.tif'

for i, file in enumerate(crop_mats):
    mat_file_name = os.path.join(pth_mat, file)

    WSI_path = os.path.join(pth, WSIs[i])

    f_out_name = f'{os.path.join(os.path.join(outpth,WSIs[i][:-5]))}{file_format}'

    if os.path.exists(f_out_name):
        print(f'Skipping cropping for {WSIs[i]} ({i+1}/{len(crop_mats)})... Already exists')

    else:
        print(f'Cropping {WSIs[i]} ({i+1}/{len(crop_mats)})')

        # print(mat_file_name)
        # print(WSI_path)

        WSI_slide = openslide.OpenSlide(WSI_path)

        wsi_width, wsi_height = WSI_slide.level_dimensions[0]


        # crop_width = int(data['crop_width']*ratio)
        # crop_height = int(data['crop_height']*ratio)


        crop_width = 512
        crop_height = 512

        try:
            data = loadmat(mat_file_name)

            crop_x = int(data['crop_x']*ratio)
            crop_y = int(data['crop_y']*ratio)
        except:
            print(mat_file_name)
            data = h5py.File(mat_file_name, 'r')

            crop_x = int(data['crop_x'][()]*ratio)
            crop_y = int(data['crop_y'][()]*ratio)


        lower_x = crop_x
        upper_y = crop_y
        upper_x = crop_x + crop_width
        lower_y = crop_y - crop_height

        # print(crop_x)
        # print(crop_y)
        # print(crop_width)
        # print(crop_height)

        region = WSI_slide.read_region((crop_x, crop_y), 0, (crop_width, crop_height)) # level = 0 in middle
        region_image = Image.new('RGB', region.size)
        region_image.paste(region, (0, 0))

        region_image.save(f_out_name)
        WSI_slide.close()


Skipping cropping for fallopian_tubes_AJER376_035.ndpi (1/11)... Already exists
Skipping cropping for fallopian_tubes_AJER376_211.ndpi (2/11)... Already exists
Skipping cropping for fallopian_tubes_AJER376_235.ndpi (3/11)... Already exists
Cropping fallopian_tubes_AJER376_245.ndpi (4/11)
\\10.99.68.178\andreex\students\Vasco Fallopian tube\stardist\1x\matfiles\fallopian_tubes_AJER376_245.mat


C:\Users\Vasco\AppData\Local\Temp\ipykernel_9832\2726928402.py:43: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  crop_x = int(data['crop_x'][()]*ratio)
C:\Users\Vasco\AppData\Local\Temp\ipykernel_9832\2726928402.py:44: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  crop_y = int(data['crop_y'][()]*ratio)


Skipping cropping for fallopian_tubes_AJER376_431.ndpi (5/11)... Already exists
Skipping cropping for fallopian_tubes_AJGB283_155.ndpi (6/11)... Already exists
Skipping cropping for fallopian_tubes_AJGB283_487.ndpi (7/11)... Already exists
Skipping cropping for fallopian_tubes_AJLC170_0153.ndpi (8/11)... Already exists
Skipping cropping for fallopian_tubes_AJLC170_0729.ndpi (9/11)... Already exists
Skipping cropping for fallopian_tubes_AJLF232_0153.ndpi (10/11)... Already exists
Skipping cropping for fallopian_tubes_AJLF232_0483.ndpi (11/11)... Already exists


In [38]:
crop_x


19818

In [39]:
crop_y

20134